# Last inn relevante pakker

In [1]:
!uv pip install --system polars
!uv pip install --system polars-ds
!uv pip install --system fastexcel
!uv pip install --system seaborn
!uv pip install --system matplotlib

Using Python 3.11.2 environment at: c:\Users\Yafee Ishraq\AppData\Local\Programs\Python\Python311
Audited 1 package in 34ms
Using Python 3.11.2 environment at: c:\Users\Yafee Ishraq\AppData\Local\Programs\Python\Python311
Audited 1 package in 30ms
Using Python 3.11.2 environment at: c:\Users\Yafee Ishraq\AppData\Local\Programs\Python\Python311
Audited 1 package in 20ms
Using Python 3.11.2 environment at: c:\Users\Yafee Ishraq\AppData\Local\Programs\Python\Python311
Audited 1 package in 67ms
Using Python 3.11.2 environment at: c:\Users\Yafee Ishraq\AppData\Local\Programs\Python\Python311
Audited 1 package in 12ms


In [2]:
import polars as pl
import polars_ds as pds
import seaborn as sns
import matplotlib.pyplot as plt

c:\Users\Yafee Ishraq\AppData\Local\Programs\Python\Python311\Lib\site-packages\matplotlib\projections\__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


# Les inn data 

In [3]:
df = pl.read_excel("treningsdata.xlsx")

In [4]:
df.head()

Dato,Navn,Deltok,Avlyst trening
i64,str,str,str
20240106,"""Erlend Haukefer Hauan""","""Ja""","""Ja"""
20240106,"""Bakary Jang Konateh""","""Ja""","""Ja"""
20240106,"""Sindre Haukefer Hauan""","""Ja""","""Ja"""
20240204,"""Yafee Ishraq""","""Nei""","""Ja"""
20240204,"""Erlend Haukefer Hauan""","""Nei""","""Ja"""


Databearbehandling

In [5]:
# Fikser på dato kolonne
df = (df
.with_columns(pl.col('Dato').cast(pl.String).str.slice(0,4).alias('År'),
              pl.col('Dato').cast(pl.String).str.slice(4,2).alias('Måned'),
              pl.col('Dato').cast(pl.String).str.slice(6,2).alias('Dag')
              )

# Gjør dag om k
.with_columns(pl.concat_str(['År', 'Måned', 'Dag'], separator='-').cast(pl.Date).alias('Dato'),
              pl.col('Måned','Dag').cast(pl.Int64)
              )
)


# Mapper måned nr til navn
# Bruk when-then-otherwise for å mappe månedstall til navn
df = (df
    .with_columns(pl.col("Måned").alias('Måned nr'))
    .with_columns(
     pl.when(pl.col("Måned") == 1).then(pl.lit("Januar"))
     .when(pl.col("Måned") == 2).then(pl.lit("Februar"))
     .when(pl.col("Måned") == 3).then(pl.lit("Mars"))
     .when(pl.col("Måned") == 4).then(pl.lit("April"))
     .when(pl.col("Måned") == 5).then(pl.lit("Mai"))
     .when(pl.col("Måned") == 6).then(pl.lit("Juni"))
     .when(pl.col("Måned") == 7).then(pl.lit("Juli"))
     .when(pl.col("Måned") == 8).then(pl.lit("August"))
     .when(pl.col("Måned") == 9).then(pl.lit("September"))
     .when(pl.col("Måned") == 10).then(pl.lit("Oktober"))
     .when(pl.col("Måned") == 11).then(pl.lit("November"))
     .when(pl.col("Måned") == 12).then(pl.lit("Desember"))
     .otherwise(pl.lit("Ukjent"))
     .alias("Måned")
    )
)

In [6]:
df.select('Måned nr', 'Måned').unique().sort(by ='Måned nr',descending=False)['Måned'].to_list()

['Januar', 'Februar', 'Mars', 'April', 'Mai', 'Juni']

In [7]:
df.filter(pl.col('Navn')=="Adrian S.Larsen")

Dato,Navn,Deltok,Avlyst trening,År,Måned,Dag,Måned nr
date,str,str,str,str,str,i64,i64
2024-02-17,"""Adrian S.Larsen""","""Ja""","""Nei""","""2024""","""Februar""",17,2
2024-02-24,"""Adrian S.Larsen""","""Ja""","""Nei""","""2024""","""Februar""",24,2
2024-03-02,"""Adrian S.Larsen""","""Nei""","""Ja""","""2024""","""Mars""",2,3
2024-03-08,"""Adrian S.Larsen""","""Nei""","""Nei""","""2024""","""Mars""",8,3


In [8]:
(df
.filter(pl.col('Avlyst trening')=='Nei',
        pl.col('Deltok')=='Ja')
.group_by('Navn')
.agg(pl.col('Dato').n_unique().alias('Antall treninger'))
.sort(by = ['Antall treninger'],descending=True)
)

Navn,Antall treninger
str,u32
"""Kristoffer Fauskanger""",10
"""Carlos Tveit""",9
"""Jørgen Vassenden""",9
"""Sindre Jakobsen""",9
"""Sindre Haukefer Hauan""",9
…,…
"""Thomas Thorbjørnsen""",2
"""Adrian S.Larsen""",2
"""Håkon Halvorsen""",1


In [ ]:
(df
.filter(pl.col('Avlyst trening')=='Nei',
        pl.col('Deltok')=='Ja')
.select('Dato','Navn')
.unique()
.group_by('Dato')
.agg(pl.col('Navn').alias('navn_1'),
     pl.col('Navn').alias('navn_2'),
     pl.col('Navn').alias('navn_3')
)

# Finner 3 kombinasjoner
.explode('navn_1')
.explode('navn_2')
.explode('navn_3')

# Skal samle alle navnene i en liste for å telle unike navn. Skal fjerne lister som har kun ett navn        
.with_columns(pl.concat_str(['navn_1', 'navn_2', 'navn_3'], separator=',').str.split(",").alias('kombi'))

# Sorterer liste
.with_columns(pl.col('kombi').list.sort())

# Filtrerer på lister som har 3 unike navn (etter at duplikater er fjernet)
.filter(pl.col('kombi').lbist.unique().list.n_unique()==3)
.group_by('kombi')
.agg(pl.col('Dato').n_unique().alias('Antall treninger'))
.sort(by = ['Antall treninger'],descending=True)
.with_columns(pl.col('kombi').list.join(",").alias('Kombinasjon'))
)

kombi,Antall treninger,Kombinasjon
list[str],u32,str
"[""Kristoffer Fauskanger"", ""Sindre Haukefer Hauan"", ""Sindre Jakobsen""]",8,"""Kristoffer Fauskanger,Sindre H…"
"[""Jørgen Vassenden"", ""Sindre Haukefer Hauan"", ""Sindre Jakobsen""]",7,"""Jørgen Vassenden,Sindre Haukef…"
"[""Carlos Tveit"", ""Kristoffer Fauskanger"", ""Sindre Jakobsen""]",7,"""Carlos Tveit,Kristoffer Fauska…"
"[""Jørgen Vassenden"", ""Kristoffer Fauskanger"", ""Sindre Jakobsen""]",7,"""Jørgen Vassenden,Kristoffer Fa…"
"[""Jørgen Vassenden"", ""Kristoffer Fauskanger"", ""Sindre Haukefer Hauan""]",7,"""Jørgen Vassenden,Kristoffer Fa…"
…,…,…
"[""Magnus Alexander Gjerde Bevan"", ""Sindre Jakobsen"", ""Stian Nyheim""]",1,"""Magnus Alexander Gjerde Bevan,…"
"[""Carlos Tveit"", ""Jørgen Vassenden"", ""Thomas Thorbjørnsen""]",1,"""Carlos Tveit,Jørgen Vassenden,…"
"[""Jørgen Vassenden"", ""Stian Nyheim"", ""Thomas Syversen""]",1,"""Jørgen Vassenden,Stian Nyheim,…"


In [ ]:

# Antall
antall_kombinasjon = 3

# Lager antall navn expr
navn_init = [pl.col('Navn').alias(f'navn_{i+1}') for i in range(antall_kombinasjon)]

# Initialiserer datasett
df_init = (df
.filter(pl.col('Avlyst trening')=='Nei',
        pl.col('Deltok')=='Ja')
.select('Dato','Navn')
.unique()
.group_by('Dato')
.agg(*navn_init)
)





Dato,navn_1,navn_2,navn_3
i64,list[str],list[str],list[str]
20240420,"[""Sindre Jakobsen"", ""Bjørn Nilsen"", … ""Sindre Haukefer Hauan""]","[""Sindre Jakobsen"", ""Bjørn Nilsen"", … ""Sindre Haukefer Hauan""]","[""Sindre Jakobsen"", ""Bjørn Nilsen"", … ""Sindre Haukefer Hauan""]"
20240630,"[""Erlend Haukefer Hauan"", ""Yafee Ishraq"", … ""Jørgen Vassenden""]","[""Erlend Haukefer Hauan"", ""Yafee Ishraq"", … ""Jørgen Vassenden""]","[""Erlend Haukefer Hauan"", ""Yafee Ishraq"", … ""Jørgen Vassenden""]"
20240308,"[""Sindre Jakobsen"", ""Jørgen Vassenden"", … ""Kristoffer Fauskanger""]","[""Sindre Jakobsen"", ""Jørgen Vassenden"", … ""Kristoffer Fauskanger""]","[""Sindre Jakobsen"", ""Jørgen Vassenden"", … ""Kristoffer Fauskanger""]"
20240224,"[""Sindre Haukefer Hauan"", ""Bjørn Hisdal"", … ""Jørgen Vassenden""]","[""Sindre Haukefer Hauan"", ""Bjørn Hisdal"", … ""Jørgen Vassenden""]","[""Sindre Haukefer Hauan"", ""Bjørn Hisdal"", … ""Jørgen Vassenden""]"
20240317,"[""Stian Nyheim"", ""Helge Håvås"", … ""Carlos Tveit""]","[""Stian Nyheim"", ""Helge Håvås"", … ""Carlos Tveit""]","[""Stian Nyheim"", ""Helge Håvås"", … ""Carlos Tveit""]"
…,…,…,…
20240526,"[""Bakary Jang Konateh"", ""Yafee Ishraq"", … ""Kristoffer Fauskanger""]","[""Bakary Jang Konateh"", ""Yafee Ishraq"", … ""Kristoffer Fauskanger""]","[""Bakary Jang Konateh"", ""Yafee Ishraq"", … ""Kristoffer Fauskanger""]"
20240512,"[""Sindre Haukefer Hauan"", ""Yafee Ishraq""]","[""Sindre Haukefer Hauan"", ""Yafee Ishraq""]","[""Sindre Haukefer Hauan"", ""Yafee Ishraq""]"
20240609,"[""Yafee Ishraq"", ""Kristoffer Fauskanger"", … ""Stian Nyheim""]","[""Yafee Ishraq"", ""Kristoffer Fauskanger"", … ""Stian Nyheim""]","[""Yafee Ishraq"", ""Kristoffer Fauskanger"", … ""Stian Nyheim""]"
